## Importing Files

In [57]:
from IPython.display import HTML 
import pandas as pd,numpy as np

anime = pd.read_csv("anime-dataset-2023.csv")
anime.shape
#anime

(24905, 24)

## Feature Engineering

In [1]:
# Selecting important features for the recommendation system
'''
anime = anime[[
    'Score', 'Genres', 'Type', 'Episodes', 'Studios', 
    'Source', 'Rating', 'Popularity', 'Synopsis'
]]'''

anime[['Popularity', 'Score']] = anime[['Popularity', 'Score']].replace('UNKNOWN', np.nan)

# Drop rows with NaN values in 'Popularity' or 'Score' columns
anime = anime.dropna(subset=['Popularity', 'Score'])
anime.reset_index(drop=True, inplace=True)

# Display the shape of the cleaned dataset
print(anime.shape)

NameError: name 'anime' is not defined

error uploading: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 505 No host specified')))


In [59]:
#anime.head()

In [60]:
import random

# Generate a list of random indices
random_indices = random.sample(range(len(anime)), 10)

# Create a DataFrame containing the randomly selected rows
random = pd.concat([anime.iloc[index] for index in random_indices], axis=1).T

random = random.reset_index(drop=True)
random.shape

(10, 24)

In [61]:
import re
from nltk.corpus import stopwords

def preprocess_text(text):
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = cleaned_text.split()
    lowercase_text = cleaned_text.lower()
    stopwords_set = set(stopwords.words('english'))
    words = lowercase_text.split()
    filtered_words = [word for word in words if word not in stopwords_set]
    return filtered_words

Synopsis = anime['Synopsis'].apply(preprocess_text)

In [62]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

scaler = MinMaxScaler()
scaled = scaler.fit_transform(anime[['Popularity', 'Score']])
anime_rec = pd.DataFrame(data=scaled, columns=['Popularity', 'Score'])
anime_rec['Name'] = anime['Name']  # Add 'Name' column from 'anime' DataFrame

# Reorder columns if necessary
anime_rec = anime_rec[['Name', 'Popularity', 'Score']]
anime_rec.insert(loc=3, column='asu', value=anime['Synopsis'])
#anime_rec.insert(loc=3, column='asu', value=Synopsis)
anime_rec

,Name,Popularity,Score,asu
0,Cowboy Bebop,0.002189,0.951724,"Crime is timeless. By the year 2071, humanity has expanded across the galaxy, filling the surface of other planets with settlements like those on Earth. These new societies are plagued by murder, drug use, and theft, and intergalactic outlaws are hunted by a growing number of tough bounty hunters.\n\nSpike Spiegel and Jet Black pursue criminals throughout space to make a humble living. Beneath his goofy and aloof demeanor, Spike is haunted by the weight of his violent past. Meanwhile, Jet manages his own troubled memories while taking care of Spike and the Bebop, their ship. The duo is joined by the beautiful con artist Faye Valentine, odd child Edward Wong Hau Pepelu Tivrusky IV, and Ein, a bioengineered Welsh Corgi.\n\nWhile developing bonds and working to catch a colorful cast of criminals, the Bebop crew's lives are disrupted by a menace from Spike's past. As a rival's maniacal plot continues to unravel, Spike must choose between life with his newfound family or revenge for his old wounds."
1,Cowboy Bebop: Tengoku no Tobira,0.031318,0.900690,"Another day, another bounty—such is the life of the often unlucky crew of the Bebop. However, this routine is interrupted when Faye, who is chasing a fairly worthless target on Mars, witnesses an oil tanker suddenly explode, causing mass hysteria. As casualties mount due to a strange disease spreading through the smoke from the blast, a whopping three hundred million woolong price is placed on the head of the supposed perpetrator.\n\nWith lives at stake and a solution to their money problems in sight, the Bebop crew springs into action. Spike, Jet, Faye, and Edward, followed closely by Ein, split up to pursue different leads across Alba City. Through their individual investigations, they discover a cover-up scheme involving a pharmaceutical company, revealing a plot that reaches much further than the ragtag team of bounty hunters could have realized."
2,Trigun,0.012767,0.878621,"Vash the Stampede is the man with a $$60,000,000,000 bounty on his head. The reason: he's a merciless villain who lays waste to all those that oppose him and flattens entire cities for fun, garnering him the title ""The Humanoid Typhoon."" He leaves a trail of death and destruction wherever he goes, and anyone can count themselves dead if they so much as make eye contact—or so the rumors say. In actuality, Vash is a huge softie who claims to have never taken a life and avoids violence at all costs.\n\nWith his crazy doughnut obsession and buffoonish attitude in tow, Vash traverses the wasteland of the planet Gunsmoke, all the while followed by two insurance agents, Meryl Stryfe and Milly Thompson, who attempt to minimize his impact on the public. But soon, their misadventures evolve into life-or-death situations as a group of legendary assassins are summoned to bring about suffering to the trio. Vash's agonizing past will be unraveled and his morality and principles pushed to the breaking point."
3,Witch Hunter Robin,0.093486,0.744828,"Robin Sena is a powerful craft user drafted into the STNJ—a group of specialized hunters that fight deadly beings known as Witches. Though her fire power is great, she's got a lot to learn about her powers and working with her cool and aloof partner, Amon. But the truth about the Witches and herself will leave Robin on an entirely new path that she never expected!\n\n(Source: Funimation)"
4,Bouken Ou Beet,0.267066,0.702069,"It is the dark century and the people are suffering under the rule of the devil, Vandel, who is able to manipulate monsters. The Vandel Busters are a group of people who hunt these devils, and among them, the Zenon Squad is known to be the strongest busters on the continent. A young boy, Beet, dreams of joining the Zenon Squad. However, one day, as a result of Beet's fault, the Zenon squad was defeated by the devil, Beltose. The five dying busters sacrificed their life power into their five weapons,

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
anime_rec['asu'] = anime_rec['asu'].fillna('')
tfidf_matrix = tfidf.fit_transform(anime_rec['asu'])
tfidf_matrix.shape


(15692, 45853)

In [64]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(anime_rec.index, index=anime_rec['Name']).drop_duplicates()

In [65]:
indices = pd.Series(anime_rec.index, index=anime_rec['Name']).drop_duplicates()
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return anime_rec['Name'].iloc[movie_indices]

In [67]:
get_recommendations('Trigun')

3473                               Trigun: Badlands Rumble
15294                                      Trigun Stampede
15501                                             Good Day
8663                           Itsumo Kokoro ni Taiyou wo!
1328                                     Hakaima Sadamitsu
11629    Kyoto Animation Koushiki Twitter: Itsumo Arigatou
9855                                    Bonobono (TV 2016)
1237     Detective Conan OVA 06: Follow the Vanished Di...
9570                                     Aoki Seimei Hoken
4830                                   Dai Yamato Zero-gou
Name: Name, dtype: object

In [ ]:
import pickle

# assuming 'model' is the variable holding your trained model
with open('example.pkl', 'wb') as file:
    pickle.dump(my_object, file)

In [ ]:
import pickle

# load the model from the pickle file
model = pickle.load(open('model.pkl', 'rb'))